In [1]:
import copy
import numpy as np

### Load data

In [2]:
raw_data = []

with open("./data/PortlandCirc.kp", "r") as f:
    for lines in f:
        raw_data.append(lines)

In [3]:
n_line = int(raw_data[0])

if("nl" in raw_data[1]):
    raw_relationship = raw_data[2: 2 + n_line]
else:
    raw_relationship = raw_data[1 + n_line: 1 + n_line * 2]
    
raw_relationship[0: 5]

['00010110110101001\n',
 '00001001001000000\n',
 '00000001001010110\n',
 '10000110110001100\n',
 '01000001001000010\n']

In [4]:
data_raw = []

for lines in raw_relationship:
    lines = lines[: -1]
    data_raw.append([int(nums) for nums in list(lines)])

data_raw[: 5]

[[0, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0],
 [1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
 [0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0]]

### Number of Nodes

In [5]:
nums_component = len(data_raw)

print(nums_component)

17


### Number of Ties

In [6]:
nums_tie = 0

data = copy.deepcopy(data_raw)
length = len(data)

for row in range(length):
    for col in range(length):
        if(data[row][col] == 1):
            nums_tie += 1
            data[col][row] = 0
            
print(nums_tie)

41


### Number of Components

In [ ]:
def dfs_double(data, row):
    
    '''
    1. Find all the elements that connects to (1s in the rows)
    2. Find all the elements that is connected to (1s in the cols)
    '''
    
    length = len(data)
    
    for items in range(length):
        if(data[row][items] == 0):
            continue
        data[row][items] = 0
        data = dfs_double(data, row)
    
    for items in range(length):
        if(data[items][row] == 0):
            continue
        data[items][row] = 0    
        data = dfs_double(data, items)
        
    return data

In [ ]:
nums_component = 0

data = copy.deepcopy(data_raw)
length = len(data)

#get the elements that is not connected
for i in range(len(data)):
    if(1 not in data[i] and 1 not in np.array(data).T[i]):
        nums_component += 1

#use dfs to get the number of islands
for row in range(length):
    for col in range(length):
        if(data[row][col] == 1):
            nums_component += 1
            data = dfs_double(data, row)
            
            
print(nums_component)

In [ ]:
'''
#For test

import pandas as pd

test = pd.DataFrame(data_raw)
test
'''
